In [1]:
import QuantLib as ql

In [2]:
today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

In [3]:
forecast_curve = ql.RelinkableYieldTermStructureHandle()
discount_curve = ql.RelinkableYieldTermStructureHandle()

In [4]:
index = ql.Euribor6M(forecast_curve)

In [5]:
issueDate = ql.Date(13, ql.October, 2014)
maturityDate = ql.Date(13, ql.October, 2024)
schedule = ql.Schedule(issueDate, maturityDate, ql.Period(ql.Semiannual), ql.TARGET(), ql.Following, ql.Following, ql.DateGeneration.Backward, False)

In [6]:
bond = ql.FloatingRateBond(settlementDays=3, faceAmount=100, schedule=schedule, index=index, paymentDayCounter=ql.Actual360(), paymentConvention=ql.Following, fixingDays=index.fixingDays(), gearings=[],
                           spreads=[], caps=[], floors=[], inArrears=False, redemption=100.0, issueDate=issueDate)
bond.setPricingEngine(ql.DiscountingBondEngine(discount_curve))

In [7]:
forecast_curve.linkTo(ql.FlatForward(0, ql.TARGET(), 0.002, ql.Actual360()))

In [8]:
DM = ql.SimpleQuote(0.0)
discount_curve.linkTo((ql.ZeroSpreadedTermStructure(forecast_curve, ql.QuoteHandle(DM))))

In [9]:
print(bond.cleanPrice())

100.00000000000001


In [10]:
DM.setValue(0.001)
print(bond.cleanPrice())

98.99979030764418


In [11]:
def F(price):
    def _f(s):
        DM.setValue(s)
        return bond.cleanPrice() - price
    return _f

In [12]:
f = F(98.9997903076)
print(f(0.0))
print(f(0.002))

1.00020969240002
-0.9901429992548856


In [13]:
margin = ql.Brent().solve(F(99.6), 1e-8, 0.0, 1e-4)
print(margin)

0.00039870328652332745


In [14]:
DM.setValue(margin)
print(bond.cleanPrice())

99.59999988275108


In [16]:
value_date = index.valueDate(today)
maturity_date = index.maturityDate(today)
print(ql.InterestRate(margin, discount_curve.dayCounter(), ql.Continuous, ql.NoFrequency).equivalentRate(index.dayCounter(), ql.Simple, index.tenor().frequency(), value_date, maturity_date))

0.039874 % Actual/360 simple compounding


In [17]:
exerciseDate = today + ql.Period(1, ql.Years)
fixingDates = [today + ql.Period(n, ql.Months) for n in range(1, 12)]
option = ql.DiscreteAveragingAsianOption(ql.Average.Arithmetic, 0.0, 0, fixingDates, ql.PlainVanillaPayoff(ql.Option.Call, 100.0), ql.EuropeanExercise(exerciseDate))

In [19]:
sigma = ql.SimpleQuote(0.20)
riskFreeCurve = ql.FlatForward(0, ql.TARGET(), 0.01, ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

process = ql.BlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(100.0)), ql.YieldTermStructureHandle(riskFreeCurve), ql.BlackVolTermStructureHandle(volatility))

In [20]:
option.setPricingEngine(ql.MCDiscreteArithmeticAPEngine(process, "pseudorandom", requiredSamples=1000, seed=42))

In [21]:
def F(price):
    def _f(v):
        sigma.setValue(v)
        return option.NPV() - price
    return _f

In [22]:
print(ql.Brent().solve(F(5.0), 1e-8, 0.20, 1e-4))

0.20081193864526342


In [23]:
print(ql.Brent().solve(F(6.0), 1e-8, 0.20, 1e-4))

0.24362397543255399
